![ascii dreidel](media/dreidel.svg)

# Day 10 AoC

🕎 [Day 10 description](https://adventofcode.com/2022/day/10) 🕎


## Setup

In [ ]:
# imports
import os, re, sys, IPython, itertools

In [ ]:
# common helper, data import
def ans(val):
    return IPython.display.Markdown("**Answer: {}**".format(val))

data_fd = open('inputs/input-aoc-22-10.txt', 'r')
data = data_fd.read().strip().split('\n')

In [ ]:
def parse_inst(inststr):
    if inststr.startswith('add'):
        inst,v = inststr.split(' ')
        v = int(v)
    elif inststr.startswith('noop'):
        inst = inststr
        v = None
    return inst,v

def process_instructions(data):
    cycle = 1
    inst_list = []
    for strn in data:
        inst_list.append(AOCInst(strn, cycle))
        cycle += inst_list[-1].cycles_to_retire()
    return inst_list
        

class AOCInst(object):
    def __init__(self, inststr, issue_cycle):
        self._instr, self._imm = parse_inst(inststr)
        self._issue_cycle = issue_cycle
        
    def cycles_to_retire(self):
        if self._instr == 'addx':
            return 2
        elif self._instr == 'noop':
            return 1
        
    def retire_cycle(self):
        return self._issue_cycle + self.cycles_to_retire()

    def execute(self, register):
        #print("executing {} retire cycle {} reg {}".format(self._instr, self.retire_cycle(), register)) 
        if self._instr == 'addx':
            register = register + self._imm
        if self._instr == 'noop':
            pass
        #print("  (returning new register {})".format(register))
        return register
    
class AOCpu(object):
    def __init__(self, inpdata):
        self._cycle = 0
        self._register = 1
        self._input = inpdata
        self._instructions = process_instructions(self._input)
        self._instructions.reverse()
        self._retired_inst = []
        
    def process_to_cycle(self, n):
        while self._cycle < n:
            self.finish_cycle()
            
    def finish_cycle(self):
        self._cycle += 1        
        n = self._cycle
        inst = self._instructions[-1]
        if inst.retire_cycle() == n:
            self._register = inst.execute(self._register)
            self._retired_inst.append(self._instructions.pop())

    
    def signal_strength(self):
        return self._register * self._cycle
    def register(self):
        return self._register

## Part 1

In [ ]:
cpu = AOCpu(data)

In [ ]:
p = 0
for i in [20, 60, 100, 140, 180, 220]:
    cpu.process_to_cycle(i)
    #print(cpu.signal_strength())
    p = p+cpu.signal_strength()
ans(p)

## Part 2

In [ ]:
class AOCrt(object):
    def __init__(self, data):
        self._cycle = 0
        self._lit_pixels = set()
        self.cpu = AOCpu(data)
        self._sprite_location = self.cpu.register()
        
    def process_to_cycle(self, n):
        while self._cycle < n:
            self._cycle += 1

            self.cpu.process_to_cycle(self._cycle)
            self._sprite_location = self.cpu.register()
            self._draw_at_cycle()
            
    def _draw_at_cycle(self):
        position = (self._cycle-1) % 40 
        if position >= self._sprite_location - 1 and \
           position <= self._sprite_location + 1:
            self._lit_pixels.add(self._cycle-1%240)
            
    def draw(self):
        strn = ""
        for i in range(240):
            if i in self._lit_pixels:
                strn += "#"
            else:
                strn += '.'
            if i % 40 == 39:
                strn += '\n'
        return strn.strip()
    
            
        

In [ ]:
crt = AOCrt(data)
crt.process_to_cycle(240)
strn = crt.draw()
print(strn)

In [ ]:
ans('ELPLZGZL')

## Debug

In [ ]:
# test data
testdata = """addx 15
addx -11
addx 6
addx -3
addx 5
addx -1
addx -8
addx 13
addx 4
noop
addx -1
addx 5
addx -1
addx 5
addx -1
addx 5
addx -1
addx 5
addx -1
addx -35
addx 1
addx 24
addx -19
addx 1
addx 16
addx -11
noop
noop
addx 21
addx -15
noop
noop
addx -3
addx 9
addx 1
addx -3
addx 8
addx 1
addx 5
noop
noop
noop
noop
noop
addx -36
noop
addx 1
addx 7
noop
noop
noop
addx 2
addx 6
noop
noop
noop
noop
noop
addx 1
noop
noop
addx 7
addx 1
noop
addx -13
addx 13
addx 7
noop
addx 1
addx -33
noop
noop
noop
addx 2
noop
noop
noop
addx 8
noop
addx -1
addx 2
addx 1
noop
addx 17
addx -9
addx 1
addx 1
addx -3
addx 11
noop
noop
addx 1
noop
addx 1
noop
noop
addx -13
addx -19
addx 1
addx 3
addx 26
addx -30
addx 12
addx -1
addx 3
addx 1
noop
noop
noop
addx -9
addx 18
addx 1
addx 2
noop
noop
addx 9
noop
noop
noop
addx -1
addx 2
addx -37
addx 1
addx 3
noop
addx 15
addx -21
addx 22
addx -6
addx 1
noop
addx 2
addx 1
noop
addx -10
noop
noop
addx 20
addx 1
addx 2
addx 2
addx -6
addx -11
noop
noop
noop""".split('\n')

In [ ]:
testcpu = AOCrt(testdata)

In [ ]:
testcpu.process_to_cycle(240)

In [ ]:
testcpu.cpu.register()

In [ ]:
print(testcpu.draw())

## Notes

Spec was a fairly vague on exactly what happened when, *really* needed to code to the example.

